앙상블
    여러개의 분류기를 생성하고 그 예측을 결합함으로써 보다 정확한 최종 예측을 도출하는 기법이다.
    비정형 데이터에 딥러닝이 있다면, 정형 데이터에는 앙상블을 이용한 머신 러닝이 있다. 
    대표적으로 랜덤 포레스트와 그래디언트 부스팅 알고리즘이 있다.
    
부스팅
    앙상블 알고리즘 중 인기와 강세를 받아 많이 발전되었다.
    XGBoost
    lightGBM
    
앙상블 학습의 유형

보팅
배깅
    보팅과 배깅은 여러개의 분류기 모델이 투표를 통해 최종예측 결과를 결정하는 방식이다.
    차이점으로,
        보팅은 서로 다른 모델이 같은 데이터 셋을 가지고 학습한 뒤 예측한 것이다.
        배깅은 서로 같은 모델이 각각 전체 데이터에서 추출된 샘플링된 데이터(중복 가능)를 가지고 학습한 뒤 예측하는 것이다.
부스팅


스태깅



보팅
    하드 보팅
        각 모델의 예측값으로 최종 결정
    소프트 보팅
        각 모델의 예측 확률값을 평균 한 뒤 최종 결정

In [6]:
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
cancer = load_breast_cancer(as_frame = True)

In [8]:
cancer.data.head(2)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902


In [9]:
cancer.target.head(2)

0    0
1    0
Name: target, dtype: int32

In [10]:
cancer.target_names

array(['malignant', 'benign'], dtype='<U9')

In [11]:
# 보팅 방식을 사용한 머신러닝

# 서로 다른 모델을 생성
lr_clf = LogisticRegression(solver='liblinear')
knn_clf = KNeighborsClassifier(n_neighbors=8)
# n_neighbors 몇개의 영역으로 나눌 것이냐
vo_clf = VotingClassifier([('lr',lr_clf),('knn',knn_clf)], voting = 'soft')
# 보팅 할 모델 리스트를 넣어주고, voting = 'hard' 혹은 'soft'  선택

X_train, X_test, y_train, y_test = train_test_split(cancer.data,
                                                    cancer.target,
                                                    test_size = 0.2,
                                                    random_state =156)

vo_clf.fit(X_train,y_train)
pred = vo_clf.predict(X_test)
accuracy_score(y_test,pred)


0.956140350877193

In [12]:
models = [lr_clf, knn_clf]
for model in models :
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    model_name = model.__class__.__name__
    print(f'{model_name} 정확도 : {accuracy_score(y_test,pred)}')

LogisticRegression 정확도 : 0.9473684210526315
KNeighborsClassifier 정확도 : 0.9385964912280702


C:\Users\park\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [13]:
# 랜덤 포레스트

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
def get_new_df(old_df):
    # 컬럼이름 재 정의,  기존 콜럼 -> 기존 콜럼_1 중복 시 기존콜럼_2
    
    dup_df = pd.DataFrame(data= old_df.groupby('column_name').cumcount(),columns=['dup_cnt'])
    dup_df = dup_df.reset_index()
    new_df = pd.merge(old_df.reset_index(), dup_df, how='outer')
    # reser_index  ??? index를 기준으로 병합???
    new_df['column_name'] = new_df[['column_name','dup_cnt']].apply(lambda x:x[0]+'_'+str(x[1]) if x[1]>0 else x[0],axis=1)
    new_df.drop(columns=['index'],inplace=True)
    return new_df

def get_human_dataset() :
    feature_name_df = pd.read_csv('human_activity/features.txt',
                 sep = '\s+',
                 header = None,
                 names = ['column_index','column_name'])
    
    name_df = get_new_df(feature_name_df)
    feature_name = name_df.iloc[:,1].values.tolist()
    X_train = pd.read_csv('human_activity/train/X_train.txt',
                          sep = '\s+',
                          names = feature_name)
    X_test = pd.read_csv('human_activity/test/X_test.txt',
                          sep = '\s+',
                          names = feature_name)
    y_train = pd.read_csv('human_activity/train/y_train.txt',
                          sep = '\s+',
                          names = ['action'])
    y_test = pd.read_csv('human_activity/test/y_test.txt',
                          sep = '\s+',
                          names = ['action'])
    
    return X_train, X_test, y_train, y_test

In [16]:
X_train, X_test, y_train, y_test = get_human_dataset()

In [17]:
rf_clf = RandomForestClassifier(random_state= 0 , max_depth = 8)
# 결정트리를 배깅 한 것으로, 결정트리의 옵션과 같은게 많더,


rf_clf.fit(X_train,y_train)
pred = rf_clf.predict(X_test)
accuracy_score(y_test,pred)

# 높은 정확도를 보여준다.

C:\Users\park\AppData\Local\Temp\ipykernel_6124\3105830674.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf.fit(X_train,y_train)


0.9195792331184255

In [27]:
rf_clf = RandomForestClassifier(random_state= 0 )
# 결정트리를 배깅 한 것으로, 결정트리의 옵션과 같은게 많더,


rf_clf.fit(X_train,y_train)
pred = rf_clf.predict(X_test)
accuracy_score(y_test,pred)

# 높은 정확도를 보여준다.

C:\Users\hihey\AppData\Local\Temp\ipykernel_18932\4045777246.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf.fit(X_train,y_train)


0.9253478113335596

In [28]:
#랜덤포레스트의 파라미터 튜닝

from sklearn.model_selection import GridSearchCV

In [33]:
params = {
    'max_depth' : [8,16,24],
    'min_samples_split' : [2,8,16],
    'min_samples_leaf' : [1,6,12]
}

In [35]:
%%time
rf_clf = RandomForestClassifier(random_state=0, n_jobs = -1)
# n_jobs ??? 전체 다 쓴다

grid_cv = GridSearchCV(rf_clf,params,cv=2,n_jobs=-1)

grid_cv.fit(X_train,y_train)

C:\Users\hihey\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Wall time: 53.9 s


GridSearchCV(cv=2, estimator=RandomForestClassifier(n_jobs=-1, random_state=0),
             n_jobs=-1,
             param_grid={'max_depth': [8, 16, 24],
                         'min_samples_leaf': [1, 6, 12],
                         'min_samples_split': [2, 8, 16]})

In [36]:
grid_cv.best_params_

{'max_depth': 16, 'min_samples_leaf': 6, 'min_samples_split': 2}

In [37]:
grid_cv.best_score_

0.9164853101196953

In [38]:
rf_clf = RandomForestClassifier(random_state= 0 , max_depth = 16, min_samples_leaf = 6, min_samples_split = 2)
rf_clf.fit(X_train,y_train)
pred = rf_clf.predict(X_test)
accuracy_score(y_test,pred)


C:\Users\hihey\AppData\Local\Temp\ipykernel_18932\2805512300.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf.fit(X_train,y_train)


0.9260264675941635